### P0 Code Generator for MIPS
#### Emil Sekerinski, March 2017
#### Modified by Dom DiPasquale, Patrick Jakubowski, and Mustafa Haddara

The generated code is kept in memory and all code generation procedures continuously append to that code: procedure `genProgStart` initializes the generator, then `gen`-prefixed procedures are to be called for P0 constructs as in order in which they are recognized by a recursive descent parser, and finally procedure `genProgExit` returns the generated code in assembly language as a string in a format that can be read in by the SPIM simulator. The generation procedures are:
- `genBool`, `genInt`, `genRec`, `genArray`
- `genProgStart`, `genGlobalVars`, `genProgEntry`, `genProgExit`
- `genProcStart`, `genFormalParams`, `genLocalVars`, `genProcEntry`, `genProcExit`
- `genSelect`, `genIndex`, `genVar`, `genConst`, `genUnaryOp`, `genBinaryOp`, `genRelation`
- `genAssign`, `genActualPara`, `genCall`, `genRead`, `genWrite`, `genWriteln`
- `genSeq`, `genCond`, `genIfThen`, `genThen`, `genIfElse`, `genTarget`, `genWhile`

Errors in the code generator are reported by calling `mark` of the scanner. The data types of the symbol table are used to specify the P0 constructs for which code is to be generated.

In [ ]:
import nbimporter
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, LE, \
     GE, NOT, mark
from ST import Var, Ref, Const, Type, Proc, StdProc, Int, Bool

Following variables determine the state of the code generator:

- `curlev` is the current level of nesting of P0 procedures
- `regs` is the set of available MIPS registers for expression evaluation
- `label` is a counter for generating new labels
- `declarations` is a list of list of declared variables (from the .data section). Each list is a list of declared variables per scope. Each inner list consists of name and size of the declared variable
- `instructions` is a list of lists of triples; each list is a list of triples for the current scope. Each triple consists of three (possibly empty) strings
   - a label
   - an instruction, possibly with operands
   - a target (for branch and jump instructions)
- `compiledProcedures` is a list of post-optimization procedure mips instructions; these will be inserted into the program after we're done all of the optimizations
- `fieldVars` is a list of dicts, with the keys being the names of fields in records, and the keys being their placeholder variable names. See [`genSelect()`](#gen-select) for more details.
- `usedGlobalVars` is a single map of names of global variables to a boolean flag which is `True` if they are used and `False` otherwise.
- `globalVarSizes` is a map of name of global variable to its required size.
- `globalDeclarations` is a list of global variable initialization instructions

Procedure `genProgStart()` initializes these variables. Registers `$t0` to `$t9` are used as general-purpose registers.

In [ ]:
def genProgStart():
    global declarations, instructions, compiledProcedures, procNames, fieldVars, usedGlobalVars, globalVarSizes, globalDeclarations
    global curlev, label, vname, regs, possibleRegs, writtenText
    declarations = [ [] ]
    instructions = [ [] ]
    compiledProcedures = []
    procNames = []
    fieldVars = [ {} ]
    usedGlobalVars  = {}
    globalVarSizes = {}
    globalDeclarations = []
    curlev = 0
    label = 0
    vname = 0
    regs = {'$t0', '$t1', '$t2', '$t3', '$t4', '$t5', '$t6', '$t7', '$t8'}
    possibleRegs = regs.copy()
    writtenText = False
    putInstr('.data')

For each global variable, `genGlobalVars(sc, start)` adds the identifier to the list of global declarations and marks it as unused. The parameter `sc` contains the top scope  with all declarations parsed so far; only variable declarations from index `start` on in the top scope are considered. As MIPS instructions are not allowed to be identifiers, all variables get `_`  as suffix to avoid a name clash. We also insert an entry into the `usedGlobalVariables` map, flagging it as unused for now.

In [ ]:
def genGlobalVars(sc, start):
    for i in range(len(sc) - 1, start - 1, - 1):
        if type(sc[i]) == Var:
            sc[i].adr = sc[i].name + '_'
            declarations[curlev].append( (sc[i].adr, sc[i].tp.size) )
            usedGlobalVars[sc[i].adr] = False
            globalVarSizes[sc[i].adr] = sc[i].tp.size

Procedure `genProgEntry(ident)` takes the program's name as a parameter. Directives for marking the beginning of the main program are generated; the program's name is it not used. 

In [ ]:
def genProgEntry(ident):
    putInstr('.globl main')
    putInstr('.ent main')
    putLab('main')

Procedure `genProgExit(x)` takes parameter `x` with the result of previous `gen-` calls, generates code for exiting the program, directives for marking the end of the main program, runs all available optimizations, consolidates the various arrays used to store instructions (`declarations`, `compliedProcedures`, and `instructions`) and returns the complete assembly code.

In [ ]:
def assembly(l, i, t):
    """Convert label l, instruction i, target t to assembly format"""
    return (l + ':\t' if l else '\t') + i + (', ' + t if t else '')

def genProgExit(x):
    putInstr('li $v0, 10')
    putInstr('syscall')
    putInstr('.end main')
    runOptimizations()
    combineInstructions()
    return '\n'.join(assembly(l, i, t) for (l, i, t) in instructions[-1])

Procedure `combineInstructions` consolidates the various arrays used to store instructions (`declarations`, `compliedProcedures`, and `instructions`) into the main `instructions` array.

In [ ]:
def combineInstructions():
    prelude = [instructions[curlev][0]] + globalDeclarations + [('', '.text', '')]
    for proc in compiledProcedures:
        prelude += proc
    prelude += instructions[curlev][1:]
    instructions[curlev] = prelude

Running the available optimizations in the following order.
1. First we eliminate unused procedures
1. Then we eliminate dead stores; these `sw` instructions to memory addresses which are never read from
1. Then we eliminate variabls that are never used

In [ ]:
def runOptimizations():
    removeUnusedProcedures()
    deadStoreElimination()
    removeUnusedVariables()

<a id="remove-unused-procedures"></a>
The `removeUnusedProcedures` function process the global instruction block, looking for any subroutines. We do this by reading all of the instructions in order, looking for jump and links (`jal` instructions). If we find a `jal`, we know that the memory location of the linked procedure gets read and therefore, must be maintained.

Due to the possibility of subroutines accessing other subroutines, as we find a `jal` in the global instruction block, we must also recursively search each nested procedure instructions for additional jump and links via the `recursiveProcedureSearch` function.

Once we have finished processing the global instruction block, we can eliminate all procedures that are never accessed.

In [ ]:
def removeUnusedProcedures():
    global instructions, compiledProcedures
    currentInstructions = instructions[curlev]
    procedureNames = []
    temp = []
    for i in range(len(currentInstructions)):
        (l,ins,target) = currentInstructions[i]   
        if (ins == 'jal'):
            recursiveProcedureSearch(procedureNames, temp, target)
    compiledProcedures = temp

Similar to `removeUnusedProcedures`, the `recursiveProcedureSearch` function process the procedure instructions block, looking for any additional subroutines. We do this by reading all of the instructions of the current procedure in order, looking for jump and links (`jal` instructions). If we find a `jal`, we know that the memory location of the linked procedure also gets read and therefore, must be maintained.

Due to the possibility of subroutines accessing other subroutines, as we find a `jal` in the global instruction block, we must also recursively search each nested procedure instructions for additional jump and links via the `recursiveProcedureSearch` function.

In [ ]:
def recursiveProcedureSearch(procedureNames, temp, target):
    for procedure in range(len(compiledProcedures)):
        searchCurrentProcedure = False
        for line in range(len(compiledProcedures[procedure])):
            if (line == 2 and compiledProcedures[procedure][line][0] == target):
                if compiledProcedures[procedure][line][0] not in procedureNames:
                    procedureNames.append(compiledProcedures[procedure][line][0])
                    temp.append(compiledProcedures[procedure])
                    searchCurrentProcedure = True
            if (searchCurrentProcedure and compiledProcedures[procedure][line][1] == 'jal'):
                recursiveProcedureSearch(procedureNames, temp, compiledProcedures[procedure][line][2])

<a id="dead-store-elimination"></a>
The `deadStoreElimination` function process the global instruction block and the instructions in a procedure, looking for stores to memory locations that are unused. We do this by reading all of the instructions in reverse order, looking for writes (`sw` instructions) which are not preceding a read (`lw` instructions). If, reading in reverse order, we find a `sw` without having found a `lw`, we know that the memory location being written to never gets read from, so we can eliminate that `sw` instruction.

Due to the way this is detected, there is currently a significant problem that occur with respect to procedure calls: We cannot detect when a global variable is modified inside of one procedure and then read inside of another (or in the global scope)

In [ ]:
def deadStoreElimination():
    global compiledProcedures
    instructions[curlev] = deadStoreEliminationFromBlock(instructions[curlev])

Dead store elimination looks for `sw` instructions not followed by a `lw` instruction from the same reg. 

1. We go through all of the instructions in reverse order. 
1. If we have an instruction which is not a `sw` or `lw`, we keep it (add it to the `result` list). 
1. If it is a `lw` instruction, we determine the location being loaded from and store in the `readAddresses` dict
1. If it is a `sw` instruction, we determine the location being stored to. 
    - Then we check to see if that location is in the `readAddresses` dict, and is not one of `$ra` or `fp`; writes to these registers are never removed.
    - If so, we have encountered a read from this address already, so we need to keep the instruction
    - Otherwise, we can drop the instruction
1. Finally, we reverse the `result` list and return it

In [ ]:
def deadStoreEliminationFromBlock(block, insideProc=False):
    ignoreRegs = ['$ra', '$fp', '$sp']
    readAddresses = {}
    result = []
    for elem in block[::-1]:
        (_,ins,_) = elem
        # assuming all targets for branch/jump instructions are labels
        if ins.startswith('lw'):
            # this is a read
            t,reg,loc = ins.split()
            readAddresses[loc] = True
            result.append(elem)
        elif ins.startswith('sw'):
            # this is a write
            t,reg,loc = ins.split()
            # if we're writing here and we've read from it before, keep the instruction
            # or if it's a global variable and we're inside a procedure
            # otherwise we discard it
            cont = False
            for ir in ignoreRegs:
                if ir in loc:
                    result.append(elem)
                    cont = True
                    break
            if cont:
                continue
            if (loc in readAddresses and readAddresses[loc]) or (loc in usedGlobalVars and insideProc):
                result.append(elem)
                readAddresses[loc] = False
        else:
            result.append(elem)
    return result[::-1]

<a id="remove-unused-variables"></a>
The `removeUnusedVariables` function detects unused variables and removes their declarations from the declarations list. Every time we encounter a variable being used, we check if it is a global variable. If so, we mark that global variable as being used. If not, we add it to the list of used variable declarations for the current scope.

In [ ]:
def removeUnusedVariables():
    global declarations, usedGlobalVars, globalDeclarations
    currentInstructions = instructions[curlev]
    temp = []
    temp0 = []
    completed = []
    realAddrs = {}
    s = 0
    for i in range(len(currentInstructions)):
        (l,ins,target) = currentInstructions[i]
        for name in usedGlobalVars:
            if name in ins or (type(target) == str and name in target):
                if not usedGlobalVars[name]:
                    usedGlobalVars[name] = True
                    temp0.append( genDataDecl( (name, globalVarSizes[name]) ) )
        for decl in declarations[curlev]:
            name = decl[0]
            size = decl[1]
            if name in completed: continue
            if name in ins or (type(target) == str and name in target):
                completed.append(name)
                s = s + size
                realAddrs[name] = -s - 8
                temp.append( genProcDecl(decl) )
    declarations[curlev] = temp
    globalDeclarations = globalDeclarations + temp0
    return realAddrs,s

The `genDataDecl` function takes a placeholder variable declaration consisting of a tuple of `(name, size)` and translates it into the appropriate `.space` directive to preceed the program.

The `genProcDecl` function accepts the same placeholder declaration but does nothing with it. We process procedure declarations later.

In [ ]:
def genDataDecl(decl):
    return (decl[0], '.space ' + str(decl[1]), '')

def genProcDecl(decl):
    return decl

The `processProcInstructions` method accepts a dictionary with string keys and integer values. This dictionary contains an entry for each local variable used in the procedure, with the key being that variable's placeholder name, and the integer value being the amount of space that variable will occupy. 

The function also accepts an integer argument with the total size used by the local variables. 

With this information, we can replace the placeholder variable names with their exact stack addresses, as well as replacing the process name with the amount of stack space required for the local variables. We do this by parsing through all of the instructions for the procedure and basically executing a string replacement.

In [ ]:
def processProcInstructions(realAddrs, size):
    global instructions, procNames
    currentInstructions = instructions[curlev]
    procName = '__LOCAL_' + procNames[-1]
    localsize = str(size + 8)
    for i in range(len(currentInstructions)):
        (l,ins,target) = currentInstructions[i]
        if procName in ins:
            ins = ins.replace(procName, localsize)
        for varAddr in realAddrs:
            if type(varAddr) != str:
                continue
            varSize = str(realAddrs[varAddr])            
            if varAddr in ins:
                ins = ins.replace(varAddr, varSize)
            if type(target) == str and varAddr in target:
                target = target.replace(varAddr, varSize)
        currentInstructions[i] = (l, ins, target)
    instructions[curlev] = currentInstructions

Procedure `newVarName()` generates a new unique variable placeholder name on each call.

In [ ]:
def newVarName():
    global vname
    vname += 1
    return '__V' + str(vname)

### Common subexpression elimination<a id="common-subexpression-elimination"></a>

The basic logic for this area is as follows:

Create a list of expressions, initially empty.
Go through the current block and record every instruction, the format is:

subexpressions = [[Instruction, [lines]], [Instruction2, [lines]], [Instruction3, [lines]], ...]

(this will eventually only grab relevant ones, add and such)

This only records if there is no duplicate line, if there is the line is instead appended to the relevant lines list in the set.

At the same time, each used register used is recorded in a different list in a similar format.

```
used_registers = [[Register, [lines]], [Register2, [lines]], [Register3, [lines]], ...]
```

Quickly check that there is at least 1 entry in subexpressions with more than 1 entry in [lines], otherwise there are no common subexpressions and the optimization is redundant.  Any subexpression with a len([lines]) = 1 is removed from the list.

Not relevant for this particular compiler, but an important note:
Go through the block and check for any instances of data being stored to memory then it being loaded, this is an indication that there are not enough registers for the function of this block, and common subexpression elimination will not assist as it performs best when replacing them with register stored values, not from memory.  In fact, doing so can slow the program down in some cases.  If this occurs, the optimization does not continue.

The list of registers is crucial as it will determine what registers are free to use, what registers can be used at some points and not others, and others still that are used throughout and cannot be modified.  Since this all occurs after smart registry use optimization, we can assume that the range of lines between occurances are considered "blocked" for use.  Effectively, this means that the first and last occurance is blocked off and we cannot use that register for any other purpose during those lines. But before and after the blocked section is fair game.

This list of subexpressions is then sorted based on the len of the [lines] list for each instruction, with priority being given to those with the most (i.e. the most common subexpressions).  This ordering is important as it ensures that the best optimizations are taken in case of there being more CSEs than usable registers.

Now for each CSE:
    
Store the CSE in this scope (it may be modified later but will still be useful to store here)
For each range in the CSE (if there are occurances on 12, 18, 20, 25, you would have 3 different steps here):
    Now we must check if the CSE is actually valid in this range, and that no changes to the used variables occured. For instance:
```
        a = x + y
        b = x + y
```

x + y is a valid CSE.
```
        a = x + y + i
        b = x + y + j
```
x + y is also a valid CSE.  However, instances can occur where:
```
        a = x + y + i
        ...
        x = x + 6
        y = 20 + var
        ...

        b = x + y + j
```
x + y is NOT a valid CSE, as the value of x and y has changed.

This is important to check, as not checking could lead to incorrect code, as it would use the initial values of x + y.
We therefore check all the lines between the initial and final line to determine no changes have occured.  If any have, we skip this range.

The current range determines what register should be used.  Priority is given to those with the "best fit", the registers that are free for the shortest period before and after (but still free throughout) this range. This allows for other CSEs to potentially use registers that we otherwise may have blocked off.  If no used registers exist that are free, an usused register is used if it exists.  If none do, the CSE range is skipped and the next CSE proceeds.

Now we add a line before the first occurance of the CSE where we store the value of that subexpression to this register. We shift the line numbers of all values in both the CSE and register list appropriately.  Then, we add this register to the list (or add it's line number if it already is used elsewhere), and modify the first and second occurance of the CSE to instead use the value stored in this register.

Proceed to the next range or next CSE if no more exist.

This method will result in all (or at least as many as our registers allow) common subexpressions to be replaced with more efficient registry uses instead of recalculating the same instructions.  This method may also be looped infinitely to eliminate more complex CSEs, such as a = x + y + z + i; b = x + y + z + j, and stopped when no changes are made in a loop.

In [ ]:
'''
#This version of the function should result in any level of procedures to be optimized, but this is untested as CSEs are not
#naturally generated by the compiler.  As such, I am leaving it in as untested (but likely valid) code.

def commonSubexpressionElimination():
    global instructions, possibleRegs, regs
    
    VALID_CSE_INS = ("add", "sub", "div", "mod", "and", "or")
    commutative = {"add", "and", "or", "mul"}
    
    #These are primed with None and empty sets to make looping through each element actually do something the first cycle.  The list
    #is later sliced with these first dead elements removed
    subexpression = [[None,[]]]
    modifiedregisters = [[None,[]]]
    freeregs = possibleRegs.copy()
    
    currentInstructions = instructions[curlev]
    
    #A quick test validating that we do indeed have registers left to use for CSEE, if none there is no point in running this
    #optimization.  We also track the unused ones, that way we know which to use if/when we get to that.
    for posi in possibleRegs:
        for ins in currentInstructions:
            if posi in ins[1]:
                freeregs.remove(posi)
                break
                
    if len(freeregs) == 0:
        return
    
    print("Free registers: ")
    print(freeregs)
    
    #This first core section gets all of the possible CSEs, and every possible "unsafe" register.
    for i in range(len(currentInstructions)):
        notCSE = True
        newCSE = True
        op = None
        regIndexes = []
        
        print("-----------------")
        print("instruction number: " + str(i))
        print(currentInstructions[i])
        
        #We don't care about stores to memory, they cannot be a CSE nor do they affect the safety of CSE registers.
        if "sw" in currentInstructions[i][1]:
            print("sw detected, skipping")
            continue
        
        #I could use RE here, but I want to avoid adding a new library to the compiler.
        #For any operation that involves a register, the first will always be written to.
        for posi in possibleRegs:
            try:
                regIndexes.append(currentInstructions[i][1].index(posi))
            except ValueError:
                pass
            
        if len(regIndexes) > 0:
            regIndexes.sort()
            regIndex = regIndexes[0]
            modifiedreg = currentInstructions[i][1][regIndex:regIndex+3]
            
            print("Modified register detected: " + str(modifiedreg))
            
            for modi in modifiedregisters:
                if modi[0] == modifiedreg:
                    modi[1].append(i)
                    break
            else:
                modifiedregisters.append([modifiedreg, [i]])

                
        else:
            continue
        
        #Check if it's even a CSE valid instruction
        #Sadly can't break as need to record if/what registers are used.
        for valid in VALID_CSE_INS:
            if (valid in currentInstructions[i][1]) and ("addi" not in currentInstructions[i][1]):
                notCSE = False
                op = valid
                #print("Op was determined to be valid")
        
        #This rather neatly takes care of commutative operations.  Lists care about order, sort it if it's commutative
        regs = currentInstructions[i][1].split(",")[1:]
        if op in commutative:
            regs.sort()
        
        for j in subexpression:
            if notCSE:
                break
            else:
                if j[0] != None:
                    subregs = j[0].split(",")[1:]
                    #Now techincally, this is the operator of the current instruction, and that's not confirmed to be the same
                    #as the op we're looking at in subexpression (it may be a different valid possible CSE)
                    #But, we check that equality later, so this is good enough
                    if op in commutative:
                        subregs.sort()
                else:
                    subregs = None
            
            #Check that the registers line up and the operations are the same, add 5+10
            if (subregs == regs) and (j[0] != None) and (op in j[0]):
                #One more necessary check here, but can't add it to the condition as I need to break without appending.
                #This is the edge case of a CSE being both a CSE and a modification to the CSE.  It is incredibly difficult
                #to deal with later, so we're simply not calling it a CSE.  If it happens as the first instance, we can
                #still deal with that, so no modification to the appending in the for/else is required.
                edge = False
                for subreg in subregs:
                    if subreg in currentInstructions[i][1].split(",")[0]:
                        print("edge case hit, not adding to CSE")
                        edge = True
                if edge:
                    break
                print("adding to existing subexpression")
                j[1].append(i)
                break
        else:
            print ("New subexpression")
            subexpression.append([currentInstructions[i][1], [i]])
    
    #Slice off the initial dead items
    subexpression = subexpression[1:]
    modifiedregisters = modifiedregisters[1:]
    
    #Now that we've determined what lines of code are possible CSEs, it's time to go through and determine which are actually common
    #and not a single line, and later which also are safe. 
    for i in range(len(subexpression)):
        if (i < len(subexpression)) and (len(subexpression[i][1]) < 2):
            del subexpression[i]
    
    #There were no common subexpressions
    if len(subexpression) == 0:
        return
    
    print ("Beginning actual CSE elimination testing")
    print(modifiedregisters)
    
    #This puts the most common subexpressions at the lowest index values, this is important for limited register situations, the most
    #common CSEs should be handled first as they are likely the best candidates for efficiency increases.
    subexpression.sort(key = lambda tup: len(tup[1]), reverse = True)
    
    #We use this to track how many CSEE's we've done, no point in looking further if we only had 1 free register and used it already
    CSEEregs = 0
    for CSE in subexpression[:]:
        #Used for determining if this CSE has a safe elimination.  We only need one range for this to happen, so it's set to true
        #if any single range is found to be valid
        if CSEEregs >= len(freeregs):
            break
        
        CSEsafe = False
        #manually stepping through as changes to CSE are happening here
        
        i = 0    
        while (i< (len(CSE[1]) - 1)):
            #Relevant data for our scan
            startRange = CSE[1][i]
            endRange = CSE[1][i+1]
            CSEregs = CSE[0].split(",")[1:]
            
            #This is to make sure that our CSE isn't trying to span unsafe regions
            if startRange == "|":
                i += 2
                continue
            
            #To determine if we're going to do the CSEE in this range, we must check if it is safe to do so
            safe = True
            for reg in CSEregs:
                #Bit of a dirty way to break multiple times, but should be a bit more efficient than rerunning all these loops
                if not safe:
                    break
                for unsaferegs in modifiedregisters:
                    if not safe:
                        break
                    if reg.strip() in unsaferegs:
                        #Might still be safe, check ranges    
                        for change in unsaferegs[1]:
                            if startRange < change < endRange:
                                print("Register change in region detected!  CSE is not safe to replace!")
                                print(CSE[1])
                                safe = False
                                break
            if not safe:
                 CSE[1].insert(i+1, "|")
            else:
                CSEsafe = True
                
            i += 1
        
        if CSEsafe:
            CSEEregs += 1
                
    print(subexpression)
        
    #Ok, *finally*, we have our list of common subexpressions with breaks in unsafe regions.  We know we have the registers
    #for the most common CSEs, which we can finally start pairing with each CSE, adding lines to the code, and replacing
    #the old code.
    
    #While we're looping through the same list again, necessary as we needed "future" data to determine whether we add code or not
    lineAdditions = []
    for CSE in subexpression:
        CSEEReg = None
        CSEEIns = None
        startOffset = 0
        endOffset = 0
        for j in lineAdditions:
            print (j)
            if j <= startRange:
                startOffset += 1
            if j <= endOffset:
                endOffset += 1
        
        for i in range(len(CSE[1]) - 1):
            startRange = CSE[1][i]
            endRange = CSE[1][i+1]
            
                        
            if startRange != "|" and endRange != "|":
                if CSEEReg == None:
                    CSEEReg = freeregs.pop()
                    
                regIndexes = []
                #Reacquire the modified register in this command
                for posi in possibleRegs:
                    try:
                        regIndexes.append(CSE[0].index(posi))
                    except ValueError:
                        pass
                            
                regIndexes.sort()
                print(regIndexes)
                regIndex = regIndexes[0]
                targetreg = currentInstructions[startRange + startOffset][1][regIndex:regIndex+3]
                
                if i == 0 or CSE[1][i-1] == "|":
                    #formulate the CSEE initial instruction if we haven't already
                    if CSEEIns == None:                        
                        CSEEIns = CSE[0][:regIndex] + CSEEReg + CSE[0][regIndex+3:] 
                    
                    print("Added CSEE! Index: " + str(startRange+startOffset))
                    print(CSEEIns)

                    currentInstructions.insert(startRange + startOffset, ('', CSEEIns, ''))
                    lineAdditions.append(startRange)
                    startOffset += 1
                    endOffset += 1
                    currentInstructions[startRange+startOffset] = ('', 'add ' + targetreg +', ' + CSEEReg + ', $0', '')
                    print("Swapped CSE! Index: " + str(startRange+startOffset))
                    #print(('', 'add ' + targetreg +', ' + CSEEReg + ', $0', ''))
                
                targetreg = currentInstructions[endRange + endOffset][1][regIndex:regIndex+3]
                print("Swapped CSE!  Index: " + str(endRange + endOffset))
                currentInstructions[endRange + endOffset] = ('', 'add ' + targetreg +', ' + CSEEReg + ', $0', '')
                
    for i in currentInstructions:
        print(i)  
'''

In [ ]:
def TESTcommonSubexpressionElimination(test_instructions):
    VALID_CSE_INS = ("add", "sub", "div", "mod", "and", "or")
    commutative = {"add", "and", "or", "mul"}
    possibleRegs = {'$t0', '$t1', '$t2', '$t3', '$t4', '$t5', '$t6', '$t7', '$t8'}
    
    #These are primed with None and empty sets to make looping through each element actually do something the first cycle.  The list
    #is later sliced with these first dead elements removed
    subexpression = [[None,[]]]
    modifiedregisters = [[None,[]]]
    freeregs = possibleRegs.copy()

    '''
    #Since moved into 
    
    #Since CSE elimination relies on there being actual common subexpressions, I am inserting the instructions of the example
    #From the report here, and modifying it so that the high level CSEs are present in the low level, by reusing the same
    #registers for the program made variables (not any temporary ones made by the compiler in mips, those remain different).
    #I also shave the lw's, because they overwrite the register with the same value.  This is to better mimic an
    #environment that CSEE would run in, free up more registers for CSEE use, as well as allow CSEE to function at all.
    old_instructions = [[('', '.data', ''), ('', '.globl main', ''), ('', '.ent main', ''), ('main', '', ''),
                       ('', 'addi $t6, $0, 1', ''), ('', 'sw $t6, a_', ''), ('', 'addi $t4, $0, 2', ''),
                       ('', 'sw $t4, b_', ''), ('', 'lw $t5, a_', ''), ('', 'lw $t1, b_', ''),
                       ('', 'add $t5, $t5, $t1', ''), ('', 'sw $t5, c_', ''), ('', 'lw $t7, a_', ''), ('', 'lw $t3, b_', ''),
                       ('', 'add $t7, $t7, $t3', ''), ('', 'sw $t7, d_', ''), ('', 'lw $t0, a_', ''), ('', 'lw $t2, b_', ''),
                       ('', 'add $t0, $t0, $t2', ''), ('', 'lw $t8, c_', ''), ('', 'add $t8, $t8, $t0', ''),
                       ('', 'sw $t8, e_', ''), ('', 'lw $a0, c_', ''), ('', 'li $v0, 1', ''), ('', 'syscall', ''),
                       ('', 'lw $a0, d_', ''), ('', 'li $v0, 1', ''), ('', 'syscall', ''), ('', 'lw $a0, e_', ''),
                       ('', 'li $v0, 1', ''), ('', 'syscall', ''), ('', 'li $v0, 10', ''), ('', 'syscall', ''),
                       ('', '.end main', '')]]
    
    #The instruction set that we will use for this test:
    test_instructions = [[('', '.data', ''), ('', '.globl main', ''), ('', '.ent main', ''), ('main', '', ''),
                       ('', 'addi $t6, $0, 1', ''), ('', 'sw $t6, a_', ''), ('', 'addi $t4, $0, 2', ''),
                       ('', 'sw $t4, b_', ''),
                       ('', 'add $t5, $t4, $t6', ''), ('', 'sw $t5, c_', ''), ('', 'add $t1, $t4, $t6', ''), ('', 'add $t4, $t4, $t6', ''),
                       ('', 'add $t7, $t6, $t4', ''), ('', 'sw $t7, d_', ''), 
                       ('', 'add $t0, $t4, $t6', ''), ('', 'add $t8, $t5, $t0', ''),
                       ('', 'sw $t8, e_', ''), ('', 'lw $a0, c_', ''), ('', 'li $v0, 1', ''), ('', 'syscall', ''),
                       ('', 'lw $a0, d_', ''), ('', 'li $v0, 1', ''), ('', 'syscall', ''), ('', 'lw $a0, e_', ''),
                       ('', 'li $v0, 1', ''), ('', 'syscall', ''), ('', 'li $v0, 10', ''), ('', 'syscall', ''),
                       ('', '.end main', '')]]
    '''
    
    #currentInstructions = instructions[curlev]
    currentInstructions = test_instructions[0]
    
    #A quick test validating that we do indeed have registers left to use for CSEE, if none there is no point in running this
    #optimization.  We also track the unused ones, that way we know which to use if/when we get to that.
    for posi in possibleRegs:
        for ins in currentInstructions:
            if posi in ins[1]:
                freeregs.remove(posi)
                break
                
    if len(freeregs) == 0:
        return
    
    print("Free registers: ")
    print(freeregs)
    
    #This first core section gets all of the possible CSEs, and every possible "unsafe" register.
    for i in range(len(currentInstructions)):
        notCSE = True
        newCSE = True
        op = None
        regIndexes = []
        
        print("----------------------------")
        print("instruction number: " + str(i))
        print(currentInstructions[i])
        
        #We don't care about stores to memory, they cannot be a CSE nor do they affect the safety of CSE registers.
        if "sw" in currentInstructions[i][1]:
            print("sw detected, skipping")
            continue
        
        #I could use RE here, but I want to avoid adding a new library to the compiler.
        #For any operation that involves a register, the first will always be written to.
        for posi in possibleRegs:
            try:
                regIndexes.append(currentInstructions[i][1].index(posi))
            except ValueError:
                pass
            
        if len(regIndexes) > 0:
            regIndexes.sort()
            regIndex = regIndexes[0]
            modifiedreg = currentInstructions[i][1][regIndex:regIndex+3]
            
            print("Modified register detected: " + str(modifiedreg))
            
            for modi in modifiedregisters:
                if modi[0] == modifiedreg:
                    modi[1].append(i)
                    break
            else:
                modifiedregisters.append([modifiedreg, [i]])

                
        else:
            continue
        
        #Check if it's even a CSE valid instruction
        #Sadly can't break as need to record if/what registers are used.
        for valid in VALID_CSE_INS:
            if (valid in currentInstructions[i][1]) and ("addi" not in currentInstructions[i][1]):
                notCSE = False
                op = valid
                #print("Op was determined to be valid")
        
        #This rather neatly takes care of commutative operations.  Lists care about order, sort it if it's commutative
        regs = currentInstructions[i][1].split(",")[1:]
        if op in commutative:
            regs.sort()
        
        for j in subexpression:
            if notCSE:
                break
            else:
                if j[0] != None:
                    subregs = j[0].split(",")[1:]
                    #Now techincally, this is the operator of the current instruction, and that's not confirmed to be the same
                    #as the op we're looking at in subexpression (it may be a different valid possible CSE)
                    #But, we check that equality later, so this is good enough
                    if op in commutative:
                        subregs.sort()
                else:
                    subregs = None
            
            #Check that the registers line up and the operations are the same, add 5+10
            if (subregs == regs) and (j[0] != None) and (op in j[0]):
                #One more necessary check here, but can't add it to the condition as I need to break without appending.
                #This is the edge case of a CSE being both a CSE and a modification to the CSE.  It is incredibly difficult
                #to deal with later, so we're simply not calling it a CSE.  If it happens as the first instance, we can
                #still deal with that, so no modification to the appending in the for/else is required.
                edge = False
                for subreg in subregs:
                    if subreg in currentInstructions[i][1].split(",")[0]:
                        print("edge case hit, not adding to CSE")
                        edge = True
                if edge:
                    break
                print("adding to existing subexpression")
                j[1].append(i)
                break
        else:
            print ("New subexpression")
            subexpression.append([currentInstructions[i][1], [i]])
    
    #Slice off the initial dead items
    subexpression = subexpression[1:]
    modifiedregisters = modifiedregisters[1:]
    
    print("\n")
    
    print("Possible CSEs before culling of single instances:")
    print(subexpression)
    print("Modified Registers List:")
    print(modifiedregisters)
    
    #Now that we've determined what lines of code are possible CSEs, it's time to go through and determine which are actually common
    #and not a single line, and later which also are safe. 
    for i in range(len(subexpression)):
        if (i < len(subexpression)) and (len(subexpression[i][1]) < 2):
            del subexpression[i]
    
    #There were no common subexpressions
    if len(subexpression) == 0:
        return
    
    print("\n")
    print("Beginning CSEE safety and replacement")
    print("=======================================")
   
    
    #This puts the most common subexpressions at the lowest index values, this is important for limited register situations, the most
    #common CSEs should be handled first as they are likely the best candidates for efficiency increases.
    subexpression.sort(key = lambda tup: len(tup[1]), reverse = True)
    
    print("Sorted list of common subexpressions by amount of occurances:")
    print(subexpression)
    
    #We use this to track how many CSEE's we've done, no point in looking further if we only had 1 free register and used it already
    CSEEregs = 0
    for CSE in subexpression[:]:
        #Used for determining if this CSE has a safe elimination.  We only need one range for this to happen, so it's set to true
        #if any single range is found to be valid
        if CSEEregs >= len(freeregs):
            break
        
        CSEsafe = False
        #manually stepping through as changes to CSE are happening here
        
        i = 0    
        while (i< (len(CSE[1]) - 1)):
            #Relevant data for our scan
            startRange = CSE[1][i]
            endRange = CSE[1][i+1]
            CSEregs = CSE[0].split(",")[1:]
            
            #This is to make sure that our CSE isn't trying to span unsafe regions
            if startRange == "|":
                i += 2
                continue
            
            #To determine if we're going to do the CSEE in this range, we must check if it is safe to do so
            safe = True
            for reg in CSEregs:
                #Bit of a dirty way to break multiple times, but should be a bit more efficient than rerunning all these loops
                if not safe:
                    break
                for unsaferegs in modifiedregisters:
                    if not safe:
                        break
                    if reg.strip() in unsaferegs:
                        #Might still be safe, check ranges    
                        for change in unsaferegs[1]:
                            if startRange < change < endRange:
                                print("Register change in region detected")
                                safe = False
                                break
            if not safe:
                 CSE[1].insert(i+1, "|")
            else:
                CSEsafe = True
                
            i += 1
        
        if CSEsafe:
            CSEEregs += 1
                
    print("")
    print("Final list of CSEs with | to represent unsafe code between boundaries:")
    print(subexpression)
    print("")
        
    #Ok, *finally*, we have our list of common subexpressions with breaks in unsafe regions.  We know we have the registers
    #for the most common CSEs, which we can finally start pairing with each CSE, adding lines to the code, and replacing
    #the old code.
    
    #While we're looping through the same list again, necessary as we needed "future" data to determine whether we add code or not
    lineAdditions = []
    for CSE in subexpression:
        CSEEReg = None
        CSEEIns = None
        startOffset = 0
        endOffset = 0
        for j in lineAdditions:
            if j <= startRange:
                startOffset += 1
            if j <= endOffset:
                endOffset += 1
        
        for i in range(len(CSE[1]) - 1):
            startRange = CSE[1][i]
            endRange = CSE[1][i+1]
            
                        
            if startRange != "|" and endRange != "|":
                if CSEEReg == None:
                    CSEEReg = freeregs.pop()
                    
                regIndexes = []
                #Reacquire the modified register in this command
                for posi in possibleRegs:
                    try:
                        regIndexes.append(CSE[0].index(posi))
                    except ValueError:
                        pass
                            
                regIndexes.sort()
                regIndex = regIndexes[0]
                targetreg = currentInstructions[startRange + startOffset][1][regIndex:regIndex+3]
                
                if i == 0 or CSE[1][i-1] == "|":
                    #formulate the CSEE initial instruction if we haven't already
                    if CSEEIns == None:                        
                        CSEEIns = CSE[0][:regIndex] + CSEEReg + CSE[0][regIndex+3:] 
                    
                    print("Added CSEE! Index: " + str(startRange+startOffset))
                    print(CSEEIns)

                    currentInstructions.insert(startRange + startOffset, ('', CSEEIns, ''))
                    lineAdditions.append(startRange)
                    startOffset += 1
                    endOffset += 1
                    currentInstructions[startRange+startOffset] = ('', 'add ' + targetreg +', ' + CSEEReg + ', $0', '')
                    print("Swapped CSE! Index: " + str(startRange+startOffset))
                
                targetreg = currentInstructions[endRange + endOffset][1][regIndex:regIndex+3]
                print("Swapped CSE!  Index: " + str(endRange + endOffset))
                currentInstructions[endRange + endOffset] = ('', 'add ' + targetreg +', ' + CSEEReg + ', $0', '')
    
    returnCode = []
    for i in currentInstructions:
        returnCode.append(assembly(i[0], i[1], i[2]))
        
    return returnCode

<a id="gen-select"></a>
Procedure `genSelect(x, f)` "generates code" for `x.f`, provided `f` is in `x.fields`. Only `x.adr` is updated, no code is generated. An updated item is returned.

In essence, we treat `x.f` as an entirely different variable than any other field in `x`. In practice, two fields in `x` will typically end up next to each other in memory. However with these "optimizations" we don't guarantee that all fields in a record will occupy a contiguous memory space.

We do this by giving every field a placeholder name (via `newVarName()`) and storing the field name -> placeholder name association in the `fieldVars` dict.

Every time we enounter a field on an object, we perform a lookup for that field variable. If we can find that field variable in the dict, we have seen it before, and so we use the same placeholder name that was generated for that field. 

In [ ]:
def genSelect(x, f):
    global fieldVars
    x.tp = f.tp
    if type(x.adr) == int:
        x.adr = x.adr + f.offset
    else:
        # lookup the field name to see if we've already seen it
        fieldName = '__' + str(x) + '.' + str(f)
        if fieldName not in fieldVars[curlev]:
            tempName = newVarName()
            declarations[curlev].append( (tempName, f.tp.size) )
            fieldVars[curlev][fieldName] = tempName
            x.adr = tempName
        else:
            x.adr = fieldVars[curlev][fieldName]
    return x

Procedure `genIndex(x, y)` generates code for `x[y]`, assuming `x` is `Var` or `Ref`, `x.tp` is `Array`, and `y.tp` is `Int`. If `y` is `Const`, only `x.adr` is updated and no code is generated, otherwise code for array index calculation is generated.

We do not do the same process as with the field variables, because with an array we have the guarantee that all items in an array are in a contiguous block of memory. If we violate this promise, then all pointer semantics with respect to arrays become meaningless, and we would risk breaking the input program. As such, the `genIndex` function has not been modified.

In [ ]:
def genIndex(x, y):
    if type(y) == Const:
        offset = (y.val - x.tp.lower) * x.tp.base.size
        x.adr = x.adr + (offset if type(x.adr) == int else '+' + str(offset))
    else:
        if type(y) != Reg: y = loadItem(y)
        putOp('sub', y.reg, y.reg, x.tp.lower)
        putOp('mul', y.reg, y.reg, x.tp.base.size)
        if x.reg != R0:
            putOp('add', y.reg, x.reg, y.reg); releaseReg(x.reg)
        x.reg = y.reg
    x.tp = x.tp.base
    return x

For each local variable, `genLocalVars(sc, start)` updates the entry of the variable with the FP-relative address and returns their total size. The parameter `sc` contains the top scope with all local declarations parsed so far; only variable declarations from index `start` on in the top scope are considered.

In [ ]:
def genLocalVars(sc, start):
    s = 0 # local block size
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            s = s + sc[i].tp.size
            sc[i].adr = newVarName() # replace the address with a random name
                                     # compute the real address at the end
            name = str(sc[i].adr)
            declarations[curlev].append( (name, sc[i].tp.size) )
    return s

Procedure `genProcStart()` adds a new scope to the declarations, instructions, global variables

In [ ]:
def genProcStart():
    global curlev, declarations, instructions
    declarations.append( [] )
    instructions.append( [] )
    fieldVars.append( {} )
    curlev = curlev + 1

For the procedure prologue, we are supposed to allocate enough space for all of the local variables, but we do not yet know which of those local variables are actually used. So, we insert a placeholder which we will remove after we have processed the rest of the procedure.